In [73]:
import pandas as pd
import numpy as np
from graphviz import Digraph

In [74]:
# Gini safsızlığı ve bilgi kazancı fonksiyonları
def gini_impurity(labels):
    if len(labels) == 0:
        return 0
    unique_labels, counts = np.unique(labels, return_counts=True)
    probabilities = counts / len(labels)
    gini = 1 - np.sum(probabilities**2)
    return gini


In [75]:
def information_gain(parent_labels, left_child_labels, right_child_labels):
    parent_gini = gini_impurity(parent_labels)
    left_gini = gini_impurity(left_child_labels)
    right_gini = gini_impurity(right_child_labels)

    weighted_gini = (len(left_child_labels) / len(parent_labels)) * left_gini + \
                    (len(right_child_labels) / len(parent_labels)) * right_gini

    info_gain = parent_gini - weighted_gini
    return info_gain


In [76]:
def build_tree(df, target, depth=0):
    # Eğer tüm etiketler aynıysa, yaprağa ulaşılmıştır
    if len(df[target].unique()) == 1:
        return df[target].iloc[0]

    # Bölünecek başka özellik yoksa veya veri çerçevesi boşsa
    if len(df) == 0 or len(df.columns) == 1:
        return df[target].mode()[0]

    # En iyi bölmeyi seç
    best_gain = 0
    best_feature = None
    for feature in df.drop(columns=[target]).columns:
        unique_values = df[feature].unique()
        for value in unique_values:
            left_split = df[df[feature] == value][target].tolist()
            right_split = df[df[feature] != value][target].tolist()
            gain = information_gain(df[target].tolist(), left_split, right_split)
            if gain > best_gain:
                best_gain = gain
                best_feature = (feature, value)

    # Eğer en iyi özellik bulunamadıysa veya bilgi kazancı sıfırsa
    if best_feature is None or best_gain == 0:
        return df[target].mode()[0]

    # Veri çerçevesinden bölme işlemi sonrası kalanları çıkar
    feature, value = best_feature
    left_df = df[df[feature] == value].copy()
    right_df = df[df[feature] != value].copy()
    remaining_df = df.drop(left_df.index).drop(right_df.index)


    # Alt dalları oluştur

    left_tree = build_tree(left_df, target, depth + 1)
    right_tree = build_tree(right_df, target, depth + 1)

    return {f"{feature} == {value}": {"Yes": left_tree, "No": right_tree}}


In [77]:
def visualize_tree(tree, graph=None, parent=None, label=None):
    if graph is None:
        graph = Digraph()

    if not isinstance(tree, dict):
        graph.node(str(id(tree)), str(tree), shape="box")
        if parent is not None:
            graph.edge(str(parent), str(id(tree)), label=label)
        return graph

    for key, subtree in tree.items():
        graph.node(str(id(key)), key)
        if parent is not None:
            graph.edge(str(parent), str(id(key)), label=label)
        visualize_tree(subtree["Yes"], graph, id(key), "Yes")
        visualize_tree(subtree["No"], graph, id(key), "No")

    return graph

In [78]:
def print_rules(tree, parent_rule=""):
    if not isinstance(tree, dict):  # Eğer yaprak düğümse
        print(f"Rule: {parent_rule} => {tree}")
        return

    for key, subtree in tree.items():
        if "Yes" in subtree:
            current_rule = f"{parent_rule} AND {key}" if parent_rule else key
            print_rules(subtree["Yes"], current_rule)

        if "No" in subtree:
            negated_rule = f"{parent_rule} AND NOT {key}" if parent_rule else f"NOT {key}"
            print_rules(subtree["No"], negated_rule)

In [80]:
def predict(tree, sample):
    if not isinstance(tree, dict):
        return tree

    for key, subtree in tree.items():
        feature, value = key.split(" == ")
        if sample[feature] == value:
            return predict(subtree["Yes"], sample)
        else:
            return predict(subtree["No"], sample)


In [81]:
# Load dataset
file_path = input("Enter the path to your CSV file: ")
df = pd.read_csv(file_path)


target_col = input("Enter the target column name: ")

# Karar ağacını oluştur
# Eksik değerleri temizleme
df.dropna(subset=[target_col], inplace=True)
tree = build_tree(df, target_col)

# Karar ağacını görselleştir
graph = visualize_tree(tree)
graph.render("decision_tree2", format="png", cleanup=True)
print_rules(tree)

Enter the path to your CSV file: /content/gini1.csv
Enter the target column name: Kabul
Rule: Yaş == Genç => Hayır
Rule: NOT Yaş == Genç AND Eğitim == Yüksek AND Yaş == Orta => Hayır
Rule: NOT Yaş == Genç AND Eğitim == Yüksek AND NOT Yaş == Orta => Evet
Rule: NOT Yaş == Genç AND NOT Eğitim == Yüksek => Evet


In [83]:
# Tüm veri noktaları için tahminleri yapın ve gerçek değerlerle karşılaştırın
predictions = []
for index, row in df.iterrows():
  sample = {
      'Cinsiyet': row['Cinsiyet'],
      'Yaş': row['Yaş'],
      'Eğitim': row['Eğitim'],
  }
  predictions.append(predict(tree, sample))

# Gerçek değerler ve tahminler arasında karşılaştırma
df['Tahmin'] = predictions
print(df[['Kabul', 'Tahmin']])

# Doğruluk oranını hesapla
accuracy = (df['Kabul'] == df['Tahmin']).sum() / len(df)
print(f"Doğruluk Oranı: {accuracy}")

   Kabul Tahmin
0   Evet   Evet
1  Hayır  Hayır
2  Hayır  Hayır
3   Evet   Evet
4   Evet   Evet
5   Evet   Evet
6  Hayır  Hayır
Doğruluk Oranı: 1.0


# Full Hali

In [62]:
import pandas as pd
import numpy as np
from graphviz import Digraph

# Gini safsızlığı ve bilgi kazancı fonksiyonları
def gini_impurity(labels):
    if len(labels) == 0:
        return 0
    unique_labels, counts = np.unique(labels, return_counts=True)
    probabilities = counts / len(labels)
    gini = 1 - np.sum(probabilities**2)
    return gini

def information_gain(parent_labels, left_child_labels, right_child_labels):
    parent_gini = gini_impurity(parent_labels)
    left_gini = gini_impurity(left_child_labels)
    right_gini = gini_impurity(right_child_labels)

    weighted_gini = (len(left_child_labels) / len(parent_labels)) * left_gini + \
                    (len(right_child_labels) / len(parent_labels)) * right_gini

    info_gain = parent_gini - weighted_gini
    return info_gain

def build_tree(df, target, depth=0):
    # Eğer tüm etiketler aynıysa, yaprağa ulaşılmıştır
    if len(df[target].unique()) == 1:
        return df[target].iloc[0]

    # Bölünecek başka özellik yoksa veya veri çerçevesi boşsa
    if len(df) == 0 or len(df.columns) == 1:
        return df[target].mode()[0]

    # En iyi bölmeyi seç
    best_gain = 0
    best_feature = None
    for feature in df.drop(columns=[target]).columns:
        unique_values = df[feature].unique()
        for value in unique_values:
            left_split = df[df[feature] == value][target].tolist()
            right_split = df[df[feature] != value][target].tolist()
            gain = information_gain(df[target].tolist(), left_split, right_split)
            if gain > best_gain:
                best_gain = gain
                best_feature = (feature, value)

    # Eğer en iyi özellik bulunamadıysa veya bilgi kazancı sıfırsa
    if best_feature is None or best_gain == 0:
        return df[target].mode()[0]

    # Veri çerçevesinden bölme işlemi sonrası kalanları çıkar
    feature, value = best_feature
    left_df = df[df[feature] == value].copy()
    right_df = df[df[feature] != value].copy()
    remaining_df = df.drop(left_df.index).drop(right_df.index)


    # Alt dalları oluştur

    left_tree = build_tree(left_df, target, depth + 1)
    right_tree = build_tree(right_df, target, depth + 1)

    return {f"{feature} == {value}": {"Yes": left_tree, "No": right_tree}}

# Karar ağacını görselleştirme
def visualize_tree(tree, graph=None, parent=None, label=None):
    if graph is None:
        graph = Digraph()

    if not isinstance(tree, dict):
        graph.node(str(id(tree)), str(tree), shape="box")
        if parent is not None:
            graph.edge(str(parent), str(id(tree)), label=label)
        return graph

    for key, subtree in tree.items():
        graph.node(str(id(key)), key)
        if parent is not None:
            graph.edge(str(parent), str(id(key)), label=label)
        visualize_tree(subtree["Yes"], graph, id(key), "Yes")
        visualize_tree(subtree["No"], graph, id(key), "No")

    return graph

def print_rules(tree, parent_rule=""):
    if not isinstance(tree, dict):  # Eğer yaprak düğümse
        print(f"Rule: {parent_rule} => {tree}")
        return

    for key, subtree in tree.items():
        if "Yes" in subtree:
            current_rule = f"{parent_rule} AND {key}" if parent_rule else key
            print_rules(subtree["Yes"], current_rule)

        if "No" in subtree:
            negated_rule = f"{parent_rule} AND NOT {key}" if parent_rule else f"NOT {key}"
            print_rules(subtree["No"], negated_rule)


# Load dataset
file_path = input("Enter the path to your CSV file: ")
df = pd.read_csv(file_path)


target_col = input("Enter the target column name: ")

# Karar ağacını oluştur
# Eksik değerleri temizleme
df.dropna(subset=[target_col], inplace=True)
tree = build_tree(df, target_col)

# Karar ağacını görselleştir
graph = visualize_tree(tree)
graph.render("decision_tree", format="png", cleanup=True)
print_rules(tree)


Enter the path to your CSV file: /content/gini1.csv
Enter the target column name: Kabul


'decision_tree.png'

In [56]:
# prompt: oluşan ağaca göre verilen veri setlerini yerleşitirip kabul durumlarını kıyaslamanı istiyorum

def predict(tree, sample):
    if not isinstance(tree, dict):
        return tree

    for key, subtree in tree.items():
        feature, value = key.split(" == ")
        if sample[feature] == value:
            return predict(subtree["Yes"], sample)
        else:
            return predict(subtree["No"], sample)

# Örnek bir veri noktası oluştur
sample_data = {
    'Cinsiyet': df['Cinsiyet'].iloc[0],  # İlk verinin cinsiyetini al
    'Yaş': df['Yaş'].iloc[0],  # İlk verinin yaşını al
    'Eğitim': df['Eğitim'].iloc[0],  # İlk verinin eğitim durumunu al
}

# Tahmini al
prediction = predict(tree, sample_data)

# Tahmini yazdır
print(f"Tahmin edilen kabul durumu: {prediction}")

# Tüm veri noktaları için tahminleri yapın ve gerçek değerlerle karşılaştırın
predictions = []
for index, row in df.iterrows():
  sample = {
      'Cinsiyet': row['Cinsiyet'],
      'Yaş': row['Yaş'],
      'Eğitim': row['Eğitim'],
  }
  predictions.append(predict(tree, sample))

# Gerçek değerler ve tahminler arasında karşılaştırma
df['Tahmin'] = predictions
print(df[['Kabul', 'Tahmin']])

# Doğruluk oranını hesapla
accuracy = (df['Kabul'] == df['Tahmin']).sum() / len(df)
print(f"Doğruluk Oranı: {accuracy}")

Tahmin edilen kabul durumu: Evet
   Kabul Tahmin
0   Evet   Evet
1  Hayır  Hayır
2  Hayır  Hayır
3   Evet   Evet
4   Evet   Evet
5   Evet   Evet
6  Hayır  Hayır
Doğruluk Oranı: 1.0
